<a href="https://colab.research.google.com/github/sahug/ds-wandb/blob/main/%20WANDB%20-%20Session%203%20-%20Pipeline%20Versioning%20with%20WANDB%20Artifacts%20and%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WANDB - Session 3 - Pipeline Versioning with WANDB Artifacts and Keras.pynb**

**Artifacts**

In this notebook, we'll show you how to use W&B Artifacts to track your ML experiment pipelines.
Our sophisticated mathematical models predict the following result:

$$
{\Huge
🧪 + 🏺 = 😃}
$$


### 🤔 What are Artifacts and Why Should I Care?

An "artifact", like a Greek [amphora 🏺](https://en.wikipedia.org/wiki/Amphora),
is a produced object -- the output of a process.
In ML, the most important artifacts are _datasets_ and _models_.

And, like the [Cross of Coronado](https://indianajones.fandom.com/wiki/Cross_of_Coronado), these important artifacts belong in a museum!
That is, they should be cataloged and organized
so that you, your team, and the ML community at large can learn from them.
After all, those who don't track training are doomed to repeat it.

Using our Artifacts API, you can log `Artifact`s as outputs of W&B `Run`s or use `Artifact`s as input to `Run`s, as in this diagram,
where a training run takes in a dataset and produces a model.
 
 ![](https://gblobscdn.gitbook.com/assets%2F-Lqya5RvLedGEWPhtkjU%2F-M94QAXA-oJmE6q07_iT%2F-M94QJCXLeePzH1p_fW1%2Fsimple%20artifact%20diagram%202.png?alt=media&token=94bc438a-bd3b-414d-a4e4-aa4f6f359f21)

Since one run can use another's output as an input, Artifacts and Runs together form a directed graph -- actually, a bipartite [DAG](https://en.wikipedia.org/wiki/Directed_acyclic_graph)! -- with nodes for `Artifact`s and `Run`s
and arrows connecting `Run`s to the `Artifact`s they consume or produce.

**Install and Import**

Artifacts are part of our Python library, starting with version `0.9.2`.

Like most parts of the ML Python stack, it's available via `pip`.

In [1]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.7 MB 4.7 MB/s 
     |████████████████████████████████| 144 kB 33.2 MB/s 
     |████████████████████████████████| 181 kB 25.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (146 kB/s)
Selecting previously unselected package tree.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
import os
import wandb

**Log a Dataset**

First, let's define some Artifacts.

We start with the `Dataset`s:
- a `train`ing set, for choosing the parameters,
- a `validation` set, for choosing the hyperparameters,
- a `test`ing set, for evaluating the final model

The first cell below defines these three datasets.

In [5]:
import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import namedtuple

In [4]:
#Set random seeds
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
random.seed(hash("setting random seeds") % 2**32 -1)
np.random.seed(hash("improves reproducibility")  % 2**32 -1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 -1)

In [12]:
Dataset = namedtuple("Dataset", ["x", "y"])

#Data parameters
num_classes = 10
input_shape = (28, 28, 1)

In [7]:
def load(train_size=50_000):
  
  #Data split between train and test set
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

  #Split off a validation set for hyperparameter tuning
  x_train, x_val = x_train[:train_size], x_train[train_size:]
  y_train, y_val = y_train[:train_size], y_train[train_size:]

  training_set = Dataset(x_train, y_train)
  validation_set = Dataset(x_val, y_val)
  test_set = Dataset(x_test, y_test)

  datasets = [training_set, validation_set, test_set]

  return datasets

This sets up a pattern we'll see repeated in this example: the code to log the data as an Artifact is wrapped around the code for producing that data. In this case, the code for loading the data is separated out from the code for load_and_logging the data.

This is good practice!

In order to log these datasets as Artifacts, we just need to

1. create a Run with `wandb.init`, (L4)
2. create an `Artifact` for the dataset (L10), and
3. save and log the associated `files` (L20, L23).

In [11]:
def load_and_log():
  
  #Start a run with a type to label it and a project it can call home
  with wandb.init(project="artifact-example", job_type="load_data") as run:

    datasets = load() #seperate code for loading dataset
    names = ["training", "validation", "test"]

    #Create artifacts
    raw_data = wandb.Artifact(
        "mnist-raw", type="dataset",
        description="Raw MNIST dataset, split into train/test/val",
        metadata={"source": "keras.dataset.mnist",
                  "sizes": [len(dataset.x) for dataset in datasets]})
    
    for name, data in zip(names, datasets):
      #Store a new file in the artifact, and write something into its contents.
      with raw_data.new_file(name + ".npz", mode="wb") as file:
        np.savez(file, x=data.x, y=data.y)

    #Save the artifact to W&B
    run.log_artifact(raw_data)

load_and_log()    

wandb: Currently logged in as: sahug (use `wandb login --relogin` to force relogin)


**`wandb.init`**


When we make the `Run` that's going to produce the `Artifact`s,
we need to state which `project` it belongs to.

Depending on your workflow,
a project might be as big as `car-that-drives-itself`
or as small as `iterative-architecture-experiment-117`.

> **Rule of 👍**: if you can, keep all of the `Run`s that share `Artifact`s
inside a single project. This keeps things simple,
but don't worry -- `Artifact`s are portable across projects!

To help keep track of all the different kinds of jobs you might run,
it's useful to provide a `job_type` when making `Runs`.
This keeps the graph of your Artifacts nice and tidy.

> **Rule of 👍**: the `job_type` should be descriptive and correspond to a single step of your pipeline. Here, we separate out `load`ing data from `preprocess`ing data.

**`wandb.artifact`**

To log something as an `Artifact`, we have to first make an `Artifact` object.

Every `Artifact` has a `name` -- that's what the first argument sets.

> **Rule of 👍**: the `name` should be descriptive, but easy to remember and type --
we like to use names that are hyphen-separated and correspond to variable names in the code.

It also has a `type`. Just like `job_type`s for `Run`s,
this is used for organizing the graph of `Run`s and `Artifact`s.

> **Rule of 👍**: the `type` should be simple:
more like `dataset` or `model`
than `mnist-data-YYYYMMDD`.

You can also attach a `description` and some `metadata`, as a dictionary.
The `metadata` just needs to be serializable to JSON.

> **Rule of 👍**: the `metadata` should be as descriptive as possible.

**`artifact.new_file` and `run.log_artifact`**

Once we've made an `Artifact` object, we need to add files to it.

You read that right: _files_ with an _s_.
`Artifact`s are structured like directories,
with files and sub-directories.

> **Rule of 👍**: whenever it makes sense to do so, split the contents
of an `Artifact` up into multiple files. This will help if it comes time to scale!

We use the `new_file` method
to simultaneously write the file and attach it to the `Artifact`.
Below, we'll use the `add_file` method,
which separates those two steps.

Once we've added all of our files, we need to `log_artifact` to [wandb.ai](https://wandb.ai).

You'll notice some URLs appeared in the output,
including one for the Run page.
That's where you can view the results of the `Run`,
including any `Artifact`s that got logged.

We'll see some examples that make better use of the other components of the Run page below.

**Use a Logged Dataset Artifact**

Artifacts in W&B, unlike artifacts in museums, are designed to be used, not just stored.

Let's see what that looks like.

The cell below defines a pipeline step that takes in a raw dataset and uses it to produce a preprocessed dataset: normalized and shaped correctly.

Notice again that we split out the meat of the code, preprocess, from the code that interfaces with wandb.



In [21]:
def preprocess(dataset, normalize=True, expand_dims=True, to_categorical=True):
  
  x, y = dataset.x, dataset.y

  if normalize:
    #Scale Image to [0, 1] range
    x = x.astype("float32") / 255

  if expand_dims:
    #Make sure image has shape (28, 28, 1)
    x = np.expand_dims(x, -1)

  if to_categorical:
    #Convert class vectors to binary class metrices
    y = keras.utils.to_categorical(y, num_classes)

  return Dataset(x, y)

Now for the code that instruments this preprocess step with wandb.Artifact logging.

Note that the example below both uses an Artifact, which is new, and logs it, which is the same as the last step. Artifacts are both the inputs and the outputs of `Runs`!

We use a new `job_type`, `preprocess-data`, to make it clear that this is a different kind of job from the previous one.

In [18]:
def preprocess_and_log(steps):
  with wandb.init(project="artifact-example", job_type="load_data") as run:

    processed_data = wandb.Artifact(
        "mnist-processed", type="dataset",
        description="Processed MNIST dataset",
        metadata=steps)
    
    #Declare which artifact we will be usin
    raw_data_artifact = run.use_artifact("mnist-raw:latest")

    #If needed download the dataset
    raw_dataset = raw_data_artifact.download()

    for split in ["training", "validation", "test"]:
      raw_split = read(raw_dataset, split)
      processed_dataset = preprocess(raw_split, **steps)

      with processed_data.new_file(split + ".npz", mode="wb") as file:
        np.savez(file, x=processed_dataset.x, y=processed_dataset.y)

    run.log_artifact(processed_data)

In [19]:
def read(data_dir, split):
  filename = split + ".npz"
  data = np.load(os.path.join(data_dir, filename))

  return Dataset(x=data["x"], y=data["y"])

One thing to notice here is that the `steps` of the preprocessing
are saved with the `preprocessed_data` as `metadata`.

If you're trying to make your experiments reproducible,
capturing lots of metadata is a good idea!

Also, even though our dataset is a "`large artifact`",
the `download` step is done in much less than a second.

Expand the markdown cell below for details.

In [22]:
steps = {"normalize": True,
         "expand_dims": True,
         "to_categorical": True}

preprocess_and_log(steps)

wandb: Downloading large artifact mnist-raw:latest, 52.41MB. 3 files... Done. 0:0:0


**`run.use_artifact`**

These steps are simpler. The consumer just needs to know the name of the Artifact, plus a bit more.

That "bit more" is the alias of the particular version of the Artifact you want.

By default, the last version to be uploaded is tagged latest. Otherwise, you can pick older versions with v0/v1, etc., or you can provide your own aliases, like best or jit-script. Just like Docker Hub tags, aliases are separated from names with :, so the Artifact we want is mnist-raw:latest.

Rule of 👍: Keep aliases short and sweet. Use custom aliases like latest or best when you want an Artifact that satisifies some property

**`artifact.download`**

Now, you may be worrying about the download call. If we download another copy, won't that double the burden on memory?

Don't worry friend. Before we actually download anything, we check to see if the right version is available locally. This uses the same technology that underlies torrenting and version control with git: hashing.

As Artifacts are created and logged, a folder called artifacts in the working directory will start to fill with sub-directories, one for each Artifact. Check out its contents with !tree artifacts:

In [23]:
!tree artifacts

artifacts
└── mnist-raw:v0
    ├── test.npz
    ├── training.npz
    └── validation.npz

1 directory, 3 files


**`The Artifacts page on wandb.ai`**

Now that we've logged and used an Artifact, let's check out the Artifacts tab on the Run page.

Navigate to the Run page URL from the wandb output and select the "Artifacts" tab from the left sidebar (it's the one with the database icon, which looks like three hockey pucks stacked on top of one another).

Click a row in either the "Input Artifacts" table or in the "Output Artifacts" table, then check out the tabs ("Overview", "Metadata") to see everything logged about the Artifact.

We particularly like the "Graph View". By default, it shows a graph with the types of Artifacts and the job_types of Run as the two types of nodes, with arrows to represent consumption and production.



**Log a Model**

That's enough to see how the API for Artifacts works, but let's follow this example through to the end of the pipeline so we can see how Artifacts can improve your ML workflow.

This first cell here builds a DNN model in PyTorch -- a really simple ConvNet.

We'll start by just initializing the model, not training it. That way, we can repeat the training while keeping everything else constant.

In [24]:
def build_model(hidden_layer_sizes=[32, 64],
                kernel_sizes=[(3, 3)],
                activation="relu",
                pool_sizes=[(2, 2)],
                dropout=0.5,
                num_classes=10,
                input_shapes=(28, 28, 1)):
  
  model = keras.Sequential(
      [
       keras.Input(shape=input_shape),
       layers.Conv2D(hidden_layer_sizes[0], kernel_size=kernel_sizes[0], activation=activation),
       layers.MaxPooling2D(pool_size=pool_sizes[0]),
       layers.Conv2D(hidden_layer_sizes[-1], kernel_size=kernel_sizes[-1], activation=activation),
       layers.MaxPooling2D(pool_size=pool_sizes[-1]),
       layers.Flatten(),
       layers.Dropout(dropout),
       layers.Dense(num_classes, activation="softmax")
      ]
  )

  model.summary()

  return model

Here, we're using W&B to track the run,
and so using the [`wandb.config`](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-config/Configs_in_W%26B.ipynb)
object to store all of the hyperparameters.

The `dict`ionary version of that `config` object is a really useful piece of `metadata`, so make sure to include it!

In [30]:
def build_model_and_log(config):
  with wandb.init(project="artifact-example", job_type="initialize", config=config) as run:
    config = wandb.config

    model = build_model(**config)

    model_artifact = wandb.Artifact(
        "convnet", type="model", description="Simple AlexNet Style CNN",
        metadata=dict(config)
    )

    model.save("initalized_model.keras")

    #Another way to add a file to an Artifact
    model_artifact.add_file("initalized_model.keras")
    wandb.save("initalized_model.keras")

    run.log_artifact(model_artifact)


In [26]:
model_config = {"hidden_layer_sizes": [32, 64],
                "kernel_sizes": [3],
                "activation": "ReLU",
                "pool_sizes": [2],
                "dropout": 0.5,
                "num_classes": 10}

In [31]:
build_model_and_log(model_config)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1600)             

**`artifact.add_file`**

Instead of simultaneously writing a `new_file` and adding it to the Artifact, as in the dataset logging examples, we can also write files in one step (here, torch.save) and then add them to the Artifact in another.

Rule of 👍: use `new_file` when you can, to prevent duplication.

**Use a Logged Model Artifact**

Just like we could call use_artifact on a dataset, we can call it on our initialized_model to use it in another Run.

This time, let's train the model.

In [33]:
def train(model, training, validation, config):

  model.compile(loss="categorical_crossentropy", optimizer=config.optimizer, metrics=["accuracy"])

  callback = wandb.keras.WandbCallback(
      validation_data=(validation.x[:32], validation.y[:32]),
      input_type = "images", labels=[str(i) for i in range(10)],
      **config["callback_config"]
  )

  model.fit(training.x, training.y, validation_data=(validation.x, validation.y), batch_size=config.batch_size, epochs=config.epochs, callbacks=[callback])

We'll run two separate `Artifact`-producing `Run`s this time.

Once the first finishes `train`ing the `model`,
the `second` will consume the `trained-model` `Artifact`
by `evaluate`ing its performance on the `test_dataset`.

Also, we'll pull out the 32 examples on which the network gets the most confused --
on which the `categorical_crossentropy` is highest.

This is a good way to diagnose issues with your dataset and your model!

In [34]:
def evaluate(model, test_dataset):

    loss, accuracy = model.evaluate(test_dataset.x, test_dataset.y, verbose=1)
    highest_losses, hardest_examples, true_labels, predictions = get_hardest_k_examples(test_dataset,model)

    return loss, accuracy, highest_losses, hardest_examples, true_labels, predictions


In [35]:
from numpy.core.fromnumeric import argsort

def get_hardest_k_examples(test_dataset, model, k=32):
  class_probs = model(test_dataset.x)
  predictions = np.argmax(class_probs, axis=1)
  losses = keras.losses.categorical_crossentropy(test_dataset.y, class_probs)
  argsort_loss = np.argsort(losses)

  highest_k_losses = np.array(losses)[argsort_loss[-k:]]
  hardest_k_examples = test_dataset.x[argsort_loss[-k:]] 
  true_labels = np.argmax(test_dataset.y[argsort_loss[-k:]], axis=1)

  return highest_k_losses, hardest_k_examples, true_labels, predictions

These logging functions don't add any new `Artifact` features,
so we won't comment on them:
we're just `use`ing, `download`ing,
and `log`ging `Artifact`s.

In [60]:
def train_and_log(config):

    with wandb.init(project="artifacts-example", job_type="train", config=config) as run:
        config = wandb.config

        data = run.use_artifact('sahug/artifact-example/mnist-processed:v0', type='dataset')
        data_dir = data.download()

        training_dataset =  read(data_dir, "training")
        validation_dataset = read(data_dir, "validation")
        
        model_artifact = run.use_artifact('sahug/artifact-example/convnet:v0', type='model')
        model_dir = model_artifact.download()
        model_path = os.path.join(model_dir, "initalized_model.keras")
        model_config = model_artifact.metadata
        model = keras.models.load_model(model_path)

        config.update(model_config)
 
        train(model, training_dataset, validation_dataset, config)

        model_artifact = wandb.Artifact(
            "trained-model", type="model",
            description="NN model trained with model.fit",
            metadata=dict(model_config))

        model.save("trained_model.keras")
        model_artifact.add_file("trained_model.keras")
        wandb.save("trained_model.keras")

        run.log_artifact(model_artifact)

    return model

In [55]:
def evaluate_and_log(config=None):
    
    with wandb.init(project="artifacts-example", job_type="report", config=config) as run:
        data = run.use_artifact('sahug/artifact-example/mnist-processed:v0', type='dataset')
        data_dir = data.download()
        test_dataset = read(data_dir, "test")

        model_artifact = run.use_artifact("trained-model:latest")
        model_dir = model_artifact.download()
        model_path = os.path.join(model_dir, "trained_model.keras")
        model = keras.models.load_model(model_path)

        loss, accuracy, highest_losses, hardest_examples, true_labels, preds = evaluate(model, test_dataset)

        run.summary.update({"loss": loss, "accuracy": accuracy})

        wandb.log({"high-loss-examples":
            [wandb.Image(hard_example, caption=str(int(pred)) + "," +  str(int(label)))
             for hard_example, pred, label in zip(hardest_examples, preds, true_labels)]})

In [61]:
callback_config = {"log_weights": True, "save_model": False, "log_batch_frequency": 10}
train_config = {"batch_size": 128, "epochs": 5, "optimizer": "adam", "callback_config": callback_config}

model = train_and_log(train_config)
evaluate_and_log()

wandb: Downloading large artifact mnist-processed:v0, 212.02MB. 3 files... Done. 0:0:0


Epoch 1/5
391/391 [==============================] - 42s 106ms/step - loss: 0.3844 - accuracy: 0.8831 - _timestamp: 1648327318.0244 - _runtime: 27.0244 - val_loss: 0.0966 - val_accuracy: 0.9738
Epoch 2/5
391/391 [==============================] - 42s 107ms/step - loss: 0.1212 - accuracy: 0.9625 - _timestamp: 1648327360.0000 - _runtime: 69.0000 - val_loss: 0.0694 - val_accuracy: 0.9810
Epoch 3/5
391/391 [==============================] - 42s 106ms/step - loss: 0.0922 - accuracy: 0.9720 - _timestamp: 1648327402.0244 - _runtime: 111.0244 - val_loss: 0.0567 - val_accuracy: 0.9849
Epoch 4/5
391/391 [==============================] - 42s 107ms/step - loss: 0.0759 - accuracy: 0.9763 - _timestamp: 1648327444.0000 - _runtime: 153.0000 - val_loss: 0.0489 - val_accuracy: 0.9858
Epoch 5/5
391/391 [==============================] - 42s 107ms/step - loss: 0.0658 - accuracy: 0.9797 - _timestamp: 1648327486.0000 - _runtime: 195.0000 - val_loss: 0.0477 - val_accuracy: 0.9867



accuracy,▁▄▆▆▇▇▇▇████████████████████████████████
epoch,▁▃▅▆█
loss,█▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇██
val_loss,█▄▂▁▁
accuracy,0.97974
best_epoch,4
best_val_loss,0.04773
epoch,4
loss,0.06578
val_accuracy,0.9867


wandb: Downloading large artifact mnist-processed:v0, 212.02MB. 3 files... Done. 0:0:0


313/313 [==============================] - 3s 10ms/step - loss: 0.0396 - accuracy: 0.9873



accuracy,0.9873
loss,0.0396


**The Graph View**

Notice that we changed the `type` of the `Artifact`:
these `Run`s used a `model`, rather than `dataset`.
`Run`s that produce `model`s will be separated
from those that produce `dataset`s in the graph view on the Artifacts page.

Go check it out! As before, you'll want to head to the Run page,
select the "Artifacts" tab from the left sidebar,
pick an `Artifact`,
and then click the "Graph View" tab.

**Exploded Graphs**

You may have noticed a button labeled "Explode". Don't click that, as it will set off a small bomb underneath your humble author's desk in the W&B HQ!

Just kidding. It "explodes" the graph in a much gentler way: Artifacts and Runs become separated at the level of a single instance, rather than a `type`: the nodes are not `dataset` and `load-data`, but `dataset:mnist-raw:v1` and `load-data:sunny-smoke-1`, and so on.

This provides total insight into your pipeline, with logged metrics, metadata, and more all at your fingertips -- you're only limited by what you choose to log with us.